### Задание 1.  
#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [202]:
import requests

from bs4 import BeautifulSoup

import time

import pandas as pd

In [207]:
KEYWORDS = ['машинное обучение','python', 'парсинг','блог']

In [204]:
q = requests.get('http://habr.com/ru/all/')

sq = BeautifulSoup(q.text)

In [209]:
articles = sq.find_all('article', class_='post post_preview')

In [210]:
link_table = pd.DataFrame()

for article in articles:
    #print(article)
    preview_block = article.find('ul', class_='post__hubs inline-list')
    pb_text = preview_block.text.lower()
    #print(pb_text)
    for word in KEYWORDS:
        if word in pb_text:
            data = article.find('span', class_='post__time').text
            title = article.find('h2', class_='post__title').text.strip('\n')
            link = article.find('h2', class_='post__title')
            link = link.find('a').get('href') # вытаскивание ссылок
            #print(title, link, data)
            row = {'data':data, 'title':title, 'link':link}
            link_table = pd.concat([link_table, pd.DataFrame([row])])
            break #если слово найдено в превью, то выходим из цикла перебора слов и идем на следующий article
        else:
            continue
        

In [211]:
link_table

,data,title,link
0,сегодня в 15:00,Rust в ядре Linux,https://habr.com/ru/company/macloud/blog/562510/
0,сегодня в 14:30,Проецирование положения объектов с камеры виде...,https://habr.com/ru/company/singularis/blog/56...
0,сегодня в 14:15,Дайджест интересных материалов для мобильного ...,https://habr.com/ru/company/productivity_insid...
0,сегодня в 13:50,Как культура жителей города влияет на дизайн к...,https://habr.com/ru/company/vdsina/blog/562542/
0,сегодня в 13:09,Античные хакеры: как древнегреческий ученый Кт...,https://habr.com/ru/company/selectel/blog/561910/
0,сегодня в 12:11,Как я программировал шахматную партию против б...,https://habr.com/ru/company/ruvds/blog/562428/
0,сегодня в 11:32,Знакомство с PromQL + Cheatsheet,https://habr.com/ru/company/timeweb/blog/562378/
0,сегодня в 10:00,Реверс инжиниринг приборной панели Nissan Mura...,https://habr.com/ru/company/macloud/blog/562504/
0,сегодня в 02:59,5 самых старых нерешенных задач Математики о п...,https://habr.com/ru/company/itsoft/blog/562518/
0,вчера в 19:55,Что можно найти на испанской барахолке: новые ...,https://habr.com/ru/company/selectel/blog/561312/


### Задание 2.  
#### Обязательная часть  
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [212]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

EMAIL = ['xxx@x.ru', 'yyy@y.com']
        
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}



In [213]:
breaches = pd.DataFrame()

for email in EMAIL:
    dat=({'emailAddresses': [email]})
    avg = requests.post(URL, json=dat , headers=headers)
    #print(avg)
    #print(avg.text)
    javg = avg.json()
    for i in javg['breaches']:
        #удаление ненужных ключей из словарей, чтоб не попадали в датафрейм
        del javg['breaches'][i]['breachId']
        del javg['breaches'][i]['recordsCount']
        del javg['breaches'][i]['statistics']
        
    breaches = pd.concat([breaches, pd.DataFrame(javg['breaches'])])
    
    

    time.sleep(0.5)

In [214]:
breaches
# честно, не знаю, как правильно инвертировать столбцы и строки, чтоб итоговый датафрейм соответствовал требуемому в задании...

,16613,17609,3176,12,3,3164,2961,15,17110,17670,13094,37177,16768,41,2,3587,17009,13662,3520,3669
site,verifications.io,vk.com,parapa.mail.ru,vk.com,adobe.com,cfire.mail.ru,cdprojektred.com,imesh.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,Big data e-mail verification platform verifica...,"At some time in 2020, the Russian social netwo...","In July and August 2016, two criminals execute...",Popular Russian social networking platform VKo...,"In October of 2013, criminals penetrated Adobe...","In July and August of 2016, two criminals carr...","In March 2016, CDProjektRed.com.com's forum da...","In June 2016, a cache of over 51 million user ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
publishDate,2019-03-28T00:00:00Z,2020-05-21T00:00:00Z,2017-02-14T00:00:00Z,2016-10-29T00:00:00Z,2016-10-21T00:00:00Z,2017-02-14T00:00:00Z,2017-01-31T00:00:00Z,2016-10-23T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
site,NaN,NaN,NaN,NaN,adobe.com,NaN,NaN,imesh.com,azcentral.com,wishbone.io,myheritage.com,forums.vkmonline.com,canva.com,dropbox.com,linkedin.com,rayli.com.cn,zynga.com,netlog.com,globalreach.eu,youku.com
description,NaN,NaN,NaN,NaN,"In October of 2013, criminals penetrated Adobe...",NaN,NaN,"In June 2016, a cache of over 51 million user ...","At an unconfirmed date, online Arizona newspap...","In January 2020, the online poll website Wishb...","In October 2017, a customer database belonging...","At an unconfirmed date, the Russian-language m...","In May 2019, graphic-design site Canva's datab...",Cloud storage company Dropbox suffered a major...,"In 2012, online professional networking platfo...","On an unconfirmed date, Chinese gossip site Ra...","In September 2019, the game developer Zynga wa...",Netlog (formerly known as Facebox and Bingbox)...,"In 2016, Global Reach Technology's database wa...",Youku is a large Chinese video content company...
publishDate,NaN,NaN,NaN,NaN,2016-10-21T00:00:00Z,NaN,NaN,2016-10-23T00:00:00Z,2020-01-03T00:00:00Z,2020-05-28T00:00:00Z,2017-11-04T00:00:00Z,2021-02-11T00:00:00Z,2019-06-13T00:00:00Z,2016-10-24T00:00:00Z,2016-10-21T00:00:00Z,2017-03-01T00:00:00Z,2019-10-17T00:00:00Z,2018-02-18T00:00:00Z,2017-03-15T00:00:00Z,2017-03-24T00:00:00Z
